# 传统方法的优化
## 1.基于MMR(最大边界相关算法)的文本摘要算法实现

---

TextRank的优点在于不需要标注数据，不需要进行预训练，效果尚可。但是缺点也很明显。从其核心思想可以看出来，它挑选摘要句时会侧重于挑选那些与很多句子相似度高的句子。因此，最后会导致挑选的很多句子冗余性比较高，从而遗漏一些带有其他主题信息却“势单力薄“的句子。所以需要惩罚冗余性！

MMR是综合

MMR公式:

![alt text](mmr.png "Title")

两个参数: n: 控制摘要数量, $\lambda$:用于控制摘要的多样性

思考：怎么衡量冗余性？ 提示：冗余性: 候选句子与当前已有的摘要集合的相似性。

## 实现步骤：
1. 分句, 计算句子向量
2. 使用TextRank计算score
3. 计算各特征  
    3.1 每个句子是否包含重要词语(时间词,总结词等)  
    3.2 每个句子与标题的关联  
    3.3 位置特征:首位置和末位更重要  
    3.4 是否包含结论性的词语  
4. 汇总各路特征
5. 执行MMR算法得到摘要

In [1]:
import pandas as pd
import numpy as np
import random
import jieba
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity
import logging
import re
from sklearn.preprocessing import  MinMaxScaler

In [2]:
content = """
证券日报
02-2808:25

易会满首秀“施政理念” 聚焦“市场化”

■董文

昨天，证监会主席易会满与副主席李超、方星海等出席新闻发布会，就科创板和注册制等资本市场热点问题回答记者提问。这些问题涵盖科创板制度安排、注册制与现行核准制的关系、新股定价、退市制度、对外开放、监管改革等众多方面，据笔者观察，易会满答问的核心词是一个，即“市场化”。

首先，此次新闻发布会的整体安排就很贴近市场。

为了筹办此次发布会，易会满先于2月20日至21日赴上海证券交易所调研，与企业、证券基金机构、会计师事务所、律师事务所代表座谈，就科创板发行上市条件、定价机制、交易制度、减持制度、券商跟投、红筹企业科创板上市等问题听取意见建议。此次新闻发布会前，证监会收集了媒体记者关心关注的很多问题。

这次发布会安排上交所理事长黄红元一起回答问题，也是更加贴近市场的一个表现。从黄红元对交易所配套制度、技术准备、储备企业情况等的介绍看，确实回答了一些“解渴”的问题。

第二，易会满深入阐述了坚持市场化改革的理念。

笔者印象最深的是他对“资本市场生态系统”的看法。他说，“我们遵循市场规律，实际上就是要高度来认识按照它的规律来办事”“能够培养一个健康的生态，这比什么都重要”“好的生态的这种考验和要求，比专业显得更为重要”。
 

在我们这个市场，协调好改革、开放、发展、稳定的关系一直是监管者面临的课题。加强对各个市场主体的监管，确保市场秩序稳定、预期稳定；同时，也要激励市场主体创新发展，通过资本市场做优做强，实现优胜劣汰。平衡这些问题很不容易。

要通过深化改革、扩大开放，优化资本市场发行、上市、交易、重组、退市等各个环节的机制，营造依法办事、公平定价、诚信交易的良好市场生态。易会满强调“培养一个健康的生态，这比什么都重要”，体现了长远兼顾、内外兼修的监管理念和发展理念，值得点赞。

易会满用坚持“四个必须”和“一个合力”来描述他的监管理念。“四个必须”即必须敬畏市场、必须敬畏法治、必须敬畏专业、必须敬畏风险，“一个合力”是指“资本市场改革发展稳定离不开方方面面的支持”。其中的“敬畏市场”和“敬畏风险”都是强调尊重市场。

可见，尊重市场规律，坚持市场化改革，是易会满坚定的理念。

第三，科创板和注册制是我国资本市场市场化改革的重要步骤。

在科创板试点注册制，核心是体现市场化、法治化的具体落地，为其他板块的改革积累经验。所以，搞好科创板，不是增加一个交易板块的问题，而是深化改革的重大举措。从一定意义上说，科创板具有多层次资本市场引领者的意味，在科创板上市的企业应当具有行业创新引领者的作用。

易会满指出，“定价能力、销售能力是考验一家投行的核心竞争力”。这也是资本市场实现真正市场化的极其重要的因素。会计师事务所、律师事务所等中介机构也要提供公平、公开、专业、客观的服务。为了实现这一目标，就要引入境外成熟的机构，加大国内市场竞争的深度。

按照中央要求，要通过深化改革，打造一个规范、透明、开放、有活力、有韧性的资本市场。目前正在推进的科创板和注册制试点，是为了增强资本市场对实体经济的包容性，进一步完善支持创新的资本形成机制，根本目的是推进资本市场市场化改革。待积累经验后，向全市场推广。

目前，资本市场已经回暖，这为改革提供了良好市场条件，我们要齐心协力，坚持“严标准、稳起步”的原则，积极做好落实和应对工作，注重各市场之间的平衡，确保改革平稳启动实施。

易会满：加强与各方沟通协作 促进市场高质量发展

■本报记者 左永刚

2月27日下午3时30分，在国务院新闻办公室举行的新闻发布会上，中国证监会主席易会满，副主席李超、方星海，上海证券交易所理事长黄红元，证监会首席律师焦津洪介绍了设立科创板并试点注册制，进一步促进资本市场稳定健康发展的有关情况。

易会满表示，习近平总书记在2月22日中央政治局集体学习的重要讲话中，从深化金融供给侧结构性改革的高度对资本市场改革发展进行了进一步谋划。“这是中央对资本市场的殷切希望，是一份沉甸甸的责任要求，也是我们进一步发展资本市场的总纲领。”

易会满表示，证监会将以习近平新时代中国特色社会主义思想为指导，在国务院金融委的统一指挥协调下，主动加强与相关部委、地方党委政府和市场各方的沟通协作，努力形成工作合力，共同促进资本市场高质量发展。

坚持“四个必须” “一个合力”

对于如何贯彻落实中央的要求，如何进一步回应股民的关切，推进资本市场持续健康发展，易会满表示“主要考虑坚持‘四个必须’‘一个合力’”。

第一，必须敬畏市场，尊重规律、遵循规律，毫不动摇地推进资本市场健康发展。资本市场是现代金融体系的基础，是实体经济的“晴雨表”，不仅关系工商百业、千家万户，同时也对国家金融实力乃至参与国际金融治理能力等构成影响。

第二，必须敬畏法治，坚持依法治市、依法监管，切实保护投资者的合法权益。加强监管，保护投资者合法权益是证监会的天职。证监会将坚持增量和存量并重，把好市场入口和市场出口两道关，加强全程监管，实现市场化优胜劣汰。充分发挥资本市场并购重组主渠道作用，进一步盘活存量、提质增效。切实履行监管职责，坚决打击欺诈发行、大股东违规占用、操纵业绩、操纵并购、操纵市场、内幕交易等违法违规行为。

第三，必须敬畏专业，强化战略思维、创新思维，全面深化资本市场改革开放。一是提高宏观思维能力，贴近市场各参与方，坚持市场导向、法治导向、监管导向，加强对资本市场宏观战略问题的研究思考，加强顶层设计，增强战略定力，稳步推进重点关注问题的改革创新，在改革中、在发展中破解难题。当前重点是抓好设立科创板并试点注册制，以此带动资本市场全面深化改革。二是完善资本市场基础制度。统筹推进相关上市板块的综合改革。进一步完善交易制度，优化交易监管改革，让市场有明确预期，让投资者有公平交易的机会。进一步加强协调，打通社保基金、保险、企业年金等各类机构投资者的入市渠道，吸引更多中长期资金入市。三是以开放促改革、促发展。从机构、市场、产品等准入维度，全方位推进资本市场高水平对外开放。四是加强监管队伍专业能力建设。资本市场瞬息万变，新情况、新问题不断涌现，证监会将始终保持“本领恐慌”意识，向实践学习，向市场学习，向投资者学习，大力培养各个层级的专门家，锻造专业能力，弘扬专业精神，努力打造一支政治过硬、作风优良、精通资本市场工作的干部队伍。

第四，必须敬畏风险，坚持底线思维、运用科学方法，着力防范化解重大金融风险。防范化解金融风险特别是防止发生系统性金融风险，是金融工作的底线和根本性任务，更是监管工作的底线和根本性任务。

易会满表示，“一个合力”就是资本市场改革发展稳定离不开方方面面的支持。证监会将以习近平新时代中国特色社会主义思想为指导，在国务院金融委的统一指挥协调下，主动加强与相关部委、地方党委政府和市场各方的沟通协作，努力形成工作合力，共同促进资本市场高质量发展。

科创板不仅是一个“板” 核心在于制度创新和改革

易会满表示，设立科创板并试点注册制是党中央国务院交给我们的重大改革任务，对于支持科技创新、推动经济高质量发展、推进资本市场市场化改革和加快上海国际金融中心建设具有重要战略意义。

下一步，我们将继续扎实细致做好准备工作，推动这项重大改革有效落实落地。

易会满就科创板涉及改革的重点问题作出说明。他表示，第一，设立科创板主要目的是增强资本市场对实体经济的包容性，更好地服务具有核心技术、行业领先、有良好发展前景和口碑的企业，通过改革进一步完善支持创新的资本形成机制。第二，设立科创板是深化资本市场改革的具体举措。这次改革不仅是新设一个板块，更重要的是，坚持市场化、法治化的方向，在发行、交易、信息披露、退市等环节进行制度创新，建立健全以信息披露为中心的股票发行上市制度，发挥科创板改革试验田的作用，形成可复制可推广的经验。第三，设立科创板将坚持“严标准、稳起步”的原则，细化相关制度安排，完善风险应对预案，加强投资者教育，注重各市场之间的平衡，确保科创板并试点注册制平稳启动实施。

设立科创板并试点注册制改革，将更好地发挥市场机制作用。易会满表示，要把设立科创板并试点注册制做好，除了证监会和上交所要更好地履行职责，需要各市场参与方能够齐心协力，共同把创新的事情做好。首先发行人要充分地进行信息披露。其次，保荐机构需要在定价、承销等方面提升自身能力，需要券商、投行机构做充分准备。这是最重要的市场化因素。会计师事务所、律师事务所等中介机构也要为市场提供公平、公开、专业、客观的服务。试点注册制需要非常好的外部法律环境，所以需要进一步推进《证券法》等相关法律的修订，进一步提高违法违规成本，保护投资者的合法权益。向市场化推进，还需要建立严格的退市制度。

“如果能把上述几个因素解决好，那就能够把金融的软环境、软实力提高到一个新的水平，尤其是市场各相关参与方会出现一个脱胎换骨的变化。希望大家齐心协力把注册制推好，为今后资本市场其他板块的改革积累经验，充分发挥试验田的作用。”易会满强调。

依靠市场按市场规律办事 培养健康的资本市场生态体系

谈及间接融资市场和直接融资市场的差异，易会满坦言，“银行业和证券业有很大区别，银行业从事的间接融资主要是看现在，资本市场也是一种融资，但更多的是看未来，所以才有估值、倍数、杠杆，以及带来一系列资本市场相应的创新业务。这两个确实从出发点到最后的基本规则是不太一样的。”

“对于我来说，到证监会以后需要不断地学习，不断地研究。很多专业的问题，一方面证监会有强大的专业化团队，另一方面资本市场是一个生态体系，这需要依靠市场参与者，调动市场参与者，按市场规律来办事。我们遵循市场规律，实际上就是要高度认识并按照市场规律来办事。一个人能力是非常有限的，还是要靠我们这支专业团队、靠全市场的参与方，才能培养一个健康的资本市场生态体系。生态体系建设好了，资本市场的健康发展才有保证。”易会满最后总结说。
易会满落座 侯捷宁摄

易会满亮相：我是资本市场的一个新兵

■本报记者 苏诗钰

2月27日下午3时30分，国务院新闻办公室举行了一场新闻发布会，中国证监会新上任的主席易会满对市场近期的热点问题进行回应。

《证券日报》记者走进发布会现场时问了一句“人多吗？”核对名单的两名工作人员异口同声地说：“很多！”

据现场国新办工作人员透露，本次发布会报名记者将近百人，相比其他发布会多出很多。当记者走进发布会现场，不禁感叹“嚯！”媒体席早已座无虚席。

下午3时30分，易会满和他的小伙伴们走进发布会现场，发布会准时开始。这次发布会是易会满的“首秀”，也是他上任的第31天，换句话说，这位证监会主席上任已“满月”。

“很高兴我和我的同事今天下午和大家见面。我到证监会工作刚满一个月，我还是资本市场的一个新兵。”易会满这一番谦逊、随和的开场白，立马拉近了与现场记者的距离。

紧接着他说，从市场参与者到监管者，角色转换挑战很大，自己如履薄冰，不敢有丝毫懈怠，唯恐辜负中央的信任和大家对我的期待。当然，大家的期待也是我加强学习、做好工作的动力。

这位以“实干”而出名的证监会新主席还真的有点不一样。他的发言一丝不苟，说到自己所接触到的新东西新内容新知识的时候，非常认真。


易会满表示，这段时间，自己一方面抓紧熟悉情况，加强调查研究，另一方面，增进部门协作，共同维护资本市场平稳健康发展。

说到来证监会一个月的时间里的感触，易会满更是“金句”频频。他表示，主要有三点：第一点感受是对自己来说最需要的是要加强学习。尽管在金融行业、银行工作了35年，从基层到总行，但是相对资本市场来说自己是一个“新兵”，特别是对于当前日新月异的资本市场变化而言，自己很需要对相应的知识进行学习，新的环境需要适应，新的问题需要研究。第二点感受是证监会主席的职位好像是一个“火山口”，因为股市确实牵动着亿万投资者的心。证监会的工作直接面对市场，每一天都在做现场直播，所以不管你愿意不愿意、喜欢不喜欢，这一个月已经感受到了什么叫“火山口”，也感受到了“火山口”的味道。第三点感受是对于自己来说需要保持定力。工作当中需要不忘初心，牢记使命，坚持正确的目标和方向，把握好节奏和力度，一步一个脚印，不浮躁、不急躁，完成好上述各项任务。

整场发布会下来，易会满对科创板、注册制等多个问题进行了详细的回答解读，更是在发布会结束后被现场记者团团围住。看到记者们蜂拥而上，易会满并没有着急要走，而是依然坐在发言台上认真地听台下记者们的提问。

"""
title = "易会满首秀“施政理念” 聚焦“市场化”"


In [40]:
base_dir = '/Users/himon/Jobs/class/project-class/class1/notes/data/'

In [41]:
# 加载stopwords
with open(base_dir + 'stopwords.txt', 'r',encoding='utf8') as f:
    stopwords = set([w.strip() for w in f])

In [42]:
# 加载词向量
local_word_embeddings = {}
with open(base_dir + 'local_vocab.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        local_word_embeddings[word] = embedding

In [80]:
# 加载重要词, 根据自己的业务规则整理的词语(必须的)
people_org_set = set()
with open(base_dir + 'important_people_orgnazation.txt', 'r') as f:
    for line in f:
        people_org_set.add(line.strip())

In [81]:
people_org_set

{'中华人民共和国交通运输部',
 '中华人民共和国人力资源和社会保障部;人力资源和社会保障部;人社部',
 '中华人民共和国住房和城乡建设部',
 '中华人民共和国公安部',
 '中华人民共和国农业农村部',
 '中华人民共和国司法部;司法部',
 '中华人民共和国商务部',
 '中华人民共和国国家卫生健康委员会',
 '中华人民共和国国家发展和改革委员会;发改委',
 '中华人民共和国国家安全部',
 '中华人民共和国国家民族事务委员会',
 '中华人民共和国国防部',
 '中华人民共和国外交部;外交部',
 '中华人民共和国审计署',
 '中华人民共和国工业和信息化部',
 '中华人民共和国应急管理部',
 '中华人民共和国教育部',
 '中华人民共和国文化和旅游部',
 '中华人民共和国民政部',
 '中华人民共和国水利部',
 '中华人民共和国海关总署',
 '中华人民共和国生态环境部',
 '中华人民共和国科学技术部;科学技术部;科技部',
 '中华人民共和国自然资源部',
 '中华人民共和国财政部',
 '中华人民共和国退役军人事务部',
 '中国人民银行',
 '中国妇女报',
 '中国工程院;工程院',
 '中国日报',
 '中国民用航空局;民航总局',
 '中国气象局;气象局',
 '中国社会科学院;社科院',
 '中国科学院;科学院',
 '中国纪检监察报',
 '中国证券监督管理委员会;证监会',
 '中国银行保险监督管理委员会',
 '中国青年报',
 '中央人民广播电台',
 '中央国际广播电台',
 '中央广播电视总台',
 '中央电视台',
 '中新社',
 '习近平',
 '交通运输部;住建部',
 '人民日报',
 '住房和城乡建设部',
 '保监会主席',
 '光明日报',
 '公安部',
 '农业农村部',
 '农民日报',
 '刘士余',
 '刘明康',
 '刘鸿儒',
 '华尔街日报',
 '司法部',
 '吴定富',
 '周小川',
 '周正庆',
 '周道炯',
 '商务部',
 '国务院',
 '国务院参事室',
 '国务院发展研究中心',
 '国务院港澳事务办公室',
 '国务院研究室',
 '国家中医药管理局',
 '国家体育总局',
 '国家信访局;信访局',
 '国家医疗保障局',
 '国家卫生健康委员会;卫

In [44]:
# 分句
# 清理标点符号
def split_document(para):
    line_split = re.split(r'[|。|！|；|？|]|\n|，', para.strip())
    _seg_sents = [re.sub(r'[^\w\s]','',sent) for sent in line_split]
    _seg_sents = [sent for sent in _seg_sents  if sent != '']
    return _seg_sents

In [45]:
# 对句子进行分词
def _seg_sent(sentence):
    sentence = re.sub(r'[^\u4e00-\u9fa5]+','',sentence)
    sentence_depart = jieba.cut(sentence.strip())
    word_list = []
    for word in sentence_depart:
        if word not in stopwords:
            word_list.append(word)   
    return word_list

In [46]:
# sent to vector
def _vector(words):
    words = [w for w in words if w in local_word_embeddings]
    words_vector = np.mean([local_word_embeddings[w] for w in words], axis=0) \
        if words else np.zeros(300)
    return words_vector

# doc to matrix
def get_sentnce_vector(all_sentences_words):
    sentence_vec = np.array([_vector(words) for words in all_sentences_words])
    return sentence_vec

In [47]:
# 特征计算函数：

In [48]:
# 计算标题与所有句子的cos相似度
# cosine_similarity(a,b), b默认是none，表示a计算a中所有sample的笛卡尔积cos相似度
def get_title_similarity(sentence_vec, title_vec):
    sim_mat = cosine_similarity(sentence_vec,title_vec)
    return sim_mat

In [13]:
# 计算标题于文档中句子相似度
# 如果有两个共有的token，则赋大一些的值
def get_title_common_score(all_sentences_words, title_words):
    set_title_words = set(title_words)    
    ret = []
    for words in all_sentences_words:
        set_words = set(words)& set_title_words
        if len(set_words)>=3:
            ret.append(1.5)
        else:
            ret.append(1)
    return np.array(ret)

In [14]:
# 首位置和末位更重要
def get_position_score(sen_length):
    position_score = np.ones(sen_length)
    position_score[:3] = 2 
    position_score[-3:] = 1.5
    return position_score

In [65]:
# 是否包含时间这个特征(一般新闻的重要信息都是包含时间节点的)
def have_date(sentence):
    if re.findall('[0-9去上前明后]{1,4}年', sentence):
        return 1
    if re.findall('[0-9上个一二三四五六七八九十]{1,2}月', sentence):
        return 1
    if re.findall('[0-9上昨前]{1,4}日', sentence):
        return 1
    if re.findall('[昨|前]天', sentence):
        return 1
    return 0

# 是否有重要的词汇
def have_important_org_peo(sentence):
    for entity in people_org_set:
        if entity in sentence:
            return 1
    return 0

# 汇总此类特征
def get_entities_score(sentence):
    date_score = have_date(sentence)
    ple_org_score = have_important_org_peo(sentence)
    return 1.5 if (date_score + ple_org_score) > 0 else 1

In [66]:
# 重要词汇
def get_clue_score(sentences):
    clue_words = '总之 总而言之 综上 综上所述 一言以蔽之 概括起来说 括而言之 括而言之 要而论之 统而言之 归根到底 归根结底 简而言之'.split()
    result = []
    for sen in sentences:
        flag = 1
        for w in clue_words:
            if w in sen:
                flag = 1.4
                break
        result.append(flag)
    return np.array(result)

In [54]:
# 计算textrank
def _calc_text_rank(sentence_vec):
    # sentence_vec：[]
    sim_mat = cosine_similarity(sentence_vec) # 只有x，计算x所有sample的笛卡尔积， 
    np.fill_diagonal(sim_mat, 0) # 对角线置0
    nx_graph = nx.from_numpy_array(sim_mat)
    tol, max_iter = 1e-7, 1000
    Flag = True
    while Flag:
        try:
            pagerank_score = nx.pagerank(nx_graph, tol=tol, max_iter=max_iter)
            Flag = False
        except nx.PowerIterationFailedConvergence as e:
            print(e)
            tol *= 10
    pagerank_score = np.array([v for k, v in sorted(pagerank_score.items(), key=lambda x: x[0])])
    return pagerank_score

# 执行摘要

In [82]:
# 1.分句, 计算句子向量,
sentences = split_document(content)
all_sentences_words = [_seg_sent(sen) for sen in sentences]
sentence_vec = get_sentnce_vector(all_sentences_words) # [sen_len, dim]

In [56]:
# 2.计算textrank
pagerank_score = _calc_text_rank(sentence_vec) # (sen_len,)

In [83]:
pagerank_score

array([0.00208443, 0.00045865, 0.00361267, 0.00045865, 0.00186974,
       0.00304324, 0.0034002 , 0.00373277, 0.00249835, 0.00335949,
       0.00266972, 0.00045865, 0.00355326, 0.00254949, 0.00303953,
       0.00294827, 0.00355061, 0.00255379, 0.00328829, 0.00315739,
       0.00308384, 0.0036598 , 0.00289462, 0.00045865, 0.00373499,
       0.00344148, 0.00242616, 0.00247389, 0.00366496, 0.00280058,
       0.00303981, 0.00045865, 0.00285967, 0.00374208, 0.00304718,
       0.00320552, 0.00045865, 0.003326  , 0.00336904, 0.00246796,
       0.00222921, 0.00266398, 0.00353344, 0.00353529, 0.00325833,
       0.00045865, 0.00365306, 0.00242182, 0.00368634, 0.00317133,
       0.00393185, 0.00336072, 0.00045865, 0.00274578, 0.00301888,
       0.00331665, 0.00045865, 0.0037115 , 0.00283729, 0.00344754,
       0.00340254, 0.00045865, 0.00239069, 0.00275885, 0.00312016,
       0.00293348, 0.00360635, 0.00351677, 0.00320083, 0.0034701 ,
       0.00355609, 0.00322675, 0.00197133, 0.00310676, 0.00325

In [59]:
# 3. 计算各特征

In [67]:
# 3.1每个句子是否包含重要词语(时间词,总结词等),包含:1.5, 不包含:1
entities_score = np.array([get_entities_score(sen) for sen in sentences]) # (sen_len,)

In [69]:
# 3.2 每个句子与标题的关联
# 标题分句
title_words = _seg_sent(title)
# 标题向量化,求cos-sim
title_vec = get_sentnce_vector([title_words])
title_sim_score = get_title_similarity(sentence_vec, title_vec) # (sen_len, 1)
# 标题与句子是否有共线的词语
title_common_score = get_title_common_score(all_sentences_words, title_words) # (sen_len,)

# 最大最小归一化,因为title_sim_score[-1, 1]
scaler = MinMaxScaler((1,2))
scaler.fit(title_sim_score)
title_sim_score = scaler.transform(title_sim_score)[:,0] # 压缩,从[len, 1]-> [len]

In [70]:
# 3.3 位置特征:首位置和末位更重要
position_score = get_position_score(len(sentences)) # (sen_len,)

In [71]:
# 3.4 是否包含结论性的词语
clue_score = get_clue_score(sentences) # (120,)

In [72]:
# 4. 汇总各路特征
title_common = False
score = pagerank_score * entities_score * (title_common_score if title_common else title_sim_score) * position_score * clue_score

In [36]:
# 摘要数量
extract_num =  10

In [74]:
len(sentence_vec)

350

In [75]:
# 执行MMR算法
# 贪心算法：每次计算每个句子的mmr_score，选择最高分
n = extract_num
summary_set = []
alpha = 0.8
max_score_index = np.argmax(score) # 起始,先选择最大的一个
summary_set.append(max_score_index) 
while n > 0:
    # 冗余性
    sim_mat = cosine_similarity(sentence_vec, sentence_vec[summary_set]) 
    sim_mat = np.max(sim_mat,axis=1)  
    
    # 两个score数据对齐
    scaler = MinMaxScaler() 
    feature_score = np.array([score, sim_mat]).T 
    scaler.fit(feature_score)
    feature_score = scaler.transform(feature_score)
    [score, sim_mat] = feature_score[:,0], feature_score[:,1] 

    # 计算mmr（减去冗余性）
    mmr_score =  alpha*score - (1-alpha)*sim_mat

    mmr_score[summary_set] = -100 # 将已经是摘要的mask掉
    max_index  = np.argmax(mmr_score) # 取出来最高得分
    summary_set.append(max_index)
    n -= 1  # 控制摘要数量
    print('-----')

-----
-----
-----
-----
-----
-----
-----
-----
-----
-----


In [76]:
# 取摘要结果
summary = [sen for idx, sen in enumerate(sentences) if idx in summary_set]

In [77]:
summary

['易会满首秀施政理念 聚焦市场化',
 '易会满答问的核心词是一个',
 '易会满深入阐述了坚持市场化改革的理念',
 '易会满用坚持四个必须和一个合力来描述他的监管理念',
 '是易会满坚定的理念',
 '易会满加强与各方沟通协作 促进市场高质量发展',
 '证监会将以习近平新时代中国特色社会主义思想为指导',
 '易会满就科创板涉及改革的重点问题作出说明',
 '易会满亮相我是资本市场的一个新兵',
 '中国证监会新上任的主席易会满对市场近期的热点问题进行回应',
 '易会满并没有着急要走']

In [78]:
feature_df = pd.DataFrame({k:v for v,k in zip([score,pagerank_score,entities_score, title_sim_score, position_score,clue_score,sentences],
                              ['score','pagerank_score','entities_score', 'title_sim_score', 'position_score','clue_score','sentences'])}
                             )

In [79]:
feature_df

,score,pagerank_score,entities_score,title_sim_score,position_score,clue_score,sentences
0,0.229959,0.002084,1.0,1.280393,2.0,1,证券日报
1,0.021617,0.000459,1.0,1.000000,2.0,1,02280825
2,1.000000,0.003613,1.5,2.000000,2.0,1,易会满首秀施政理念 聚焦市场化
3,0.000000,0.000459,1.0,1.000000,1.0,1,董文
4,0.136288,0.001870,1.5,1.194582,1.0,1,昨天
...,...,...,...,...,...,...,...
345,0.291604,0.002993,1.5,1.480109,1.0,1,易会满对科创板注册制等多个问题进行了详细的回答解读
346,0.183996,0.002983,1.0,1.462365,1.0,1,更是在发布会结束后被现场记者团团围住
347,0.198294,0.002365,1.0,1.315070,1.5,1,看到记者们蜂拥而上
348,0.450177,0.002952,1.5,1.507189,1.5,1,易会满并没有着急要走


# 总结:
1. 计算摘要的过程使用的贪心算法,在没一轮选择摘要时,计算每个句子的MMR得分,放到集合中.
2. MMR方法与TextRank结合起来的.TextRank的权重得分是在句子相似度的基础上通过无监督学习得到，比直接使用相似度要更全面。当然也可以有更优秀更复杂的方案.
3. 可以根据自己的业务场景加入更多特征，简单有效！